In [1]:
import numpy as np
import scipy as sp
import pandas as pd
import pytraj as pt
import nglview as nv

import matplotlib
from matplotlib import pyplot as plt

import cphmdanalysis as cphmda

In [3]:
nRuns=5
sysName='R206H'
with open('pH_List.txt','r') as pHlistFile:
    rep_pH_vals=list(map(float,pHlistFile.readline().split(', ')))
phList=rep_pH_vals*nRuns
nReps=len(rep_pH_vals)
lambdaFileNames=['%s.rep%02g.%02g.lambda'%(sysName,irep+1,irun) \
                 for irun in np.arange(nRuns) \
                 for irep in np.arange(len(rep_pH_vals))]
lambdaFiles=[cphmda.lambda_file(filename) for filename in lambdaFileNames]

for lambda_file in lambdaFiles:
    lambda_file.compute_s_values()

pka_data=cphmda.compute_pkas(phList,lambdaFiles) #wrapped_lambda_files)

tempStruc=pt.load('%s.min.rst7'%sysName,top='%s.parm7'%sysName)
orig_struc=pt.load("../../../structures/WT.pdb")

standard_pKa_dict={
    "AS2":3.71,"GL2":4.15,"HIP":6.04,"CYS":8.14,"LYS":10.67}

tempStruc=pt.load('%s.min.rst7'%sysName,top='%s.parm7'%sysName)
pKaTable=pd.DataFrame([(tempStruc.top.residue(res-1).name,
 (tempStruc.top.residue(res-1)).original_resid,
 (orig_struc.top.residue(res-1)).original_resid+1,
  "%.3f"%(
      pka_data[ires][2]),
  "%.3f"%(
      pka_data[ires][0])) for ires,res in enumerate(lambdaFiles[0].titrreses)],
            columns=['ResName','Resid','PDB_Resid','Predicted_pKa','Hill_Coef'])
pKaTable["Standard_pKa"]=pKaTable.ResName.apply(
    lambda x: standard_pKa_dict[x])
pKaTable["Delta_pKa"]=pd.Series(pKaTable.Predicted_pKa,dtype=float)- \
    pd.Series(pKaTable.Standard_pKa,dtype=float)
pKaTable.to_csv('pKa_DataTable.csv',index=False)
goodRows=np.sum((pKaTable.dropna().Standard_pKa!='nan'))
totalRows=pKaTable.shape[0]
print('Total Number of Residues: ',
      totalRows,
      '\nNumber of succesfully titrated residues: ',
      goodRows,
      '\nNumber of unsuccesfully titrated residues: ',
      totalRows-goodRows)
pKaTable.head()

Total Number of Residues:  96 
Number of succesfully titrated residues:  73 
Number of unsuccesfully titrated residues:  23


/Users/wbotellosmith/anaconda2/envs/py3/lib/python3.7/site-packages/pandas/core/ops.py:1649: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


,ResName,Resid,PDB_Resid,Predicted_pKa,Hill_Coef,Standard_pKa,Delta_pKa
0,AS2,6,178,2.936,0.742,3.71,-0.774
1,AS2,11,183,2.946,0.516,3.71,-0.764
2,AS2,14,186,3.020,0.690,3.71,-0.690
3,HIP,15,187,7.004,0.998,6.04,0.964
4,CYS,17,189,10.098,0.379,8.14,1.958
